# Importing Packages & Setting Color Palette

In [5]:
import pandas as pd
import numpy as np
import scipy.stats
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


%matplotlib inline
sns.set(rc={'figure.figsize':(8,6)})
sns.set_style("darkgrid", {"axes.facecolor": ".93"})

palette = [ '#00BFC4' ,'#F8766D', '#0085C7', '#000000', "#FFCE54", '#A1469E']
sns.set_palette(palette)

# Just Some Words of Encouragement

In [6]:
print("Hellooo World")
print("Life is gonna be great, just you hang in there, my bruv")

Hellooo World
Life is gonna be great, just you hang in there, my bruv


# Real Shit Begins